In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
%matplotlib inline
from sympy import *
init_printing(pretty_print = False)

In [2]:
class Coisa_Fisica():
    
    def __init__(coisa, nome = None, preco = None):
        
        coisa.nome = nome
        coisa.preco = preco

In [3]:
class Substancia(Coisa_Fisica):
    
    def __init__(substancia, nome, preco, propriedades_fisqui):
        assert type(propriedades_fisqui) == dict
        
        super().__init__(nome, preco)
        for key in propriedades_fisqui:
            setattr(substancia, key, propriedades_fisqui[key])

In [4]:
class Elemento(Coisa_Fisica):
    
    def __init__(elemento, nome, simbolo, preco, ca0):
        
        super().__init__(nome, preco)
        elemento.simbolo = simbolo
        elemento.ca0 = ca0       
    
    def chutes_iniciais(elemento, n_celulas, valor_esperado_final):
        
        if isoterma.tipo == 'Extração':
            chutes = np.linspace(elemento.ca0, valor_esperado_final, n_celulas)
        else:
            if type(elemento) == Proton:
                chutes = np.linspace(valor_esperado_final, elemento.ca0, n_celulas)
            else:
                chutes = np.linspace(valor_esperado_final / isoterma.rao, elemento.con / isoterma.rao, n_celulas)
        return chutes
    
    def concentracoes_aquoso(elemento, chutes):

        cas = {'ca1': chutes[0]}
        
        for i in range(2, isoterma.n_celulas + 1):
            cas['ca' + str(i)] = chutes[i - 1]
            
        return cas

In [5]:
class Proton(Elemento):
    
    def __init__(proton, nome, simbolo, preco, ca0):
        
        super().__init__(nome, simbolo, preco, ca0)
        proton.pH_inicial = -np.log10(ca0)
        
    def pH (proton, ca):
        
        pH = -np.log10(ca)        
        return pH

In [6]:
class ETR(Elemento):
    
    def __init__(etr, nome, simbolo, preco, pureza_minima,
                 ca0, con, massa_molar_oxido, proporcao_estequiometrica, coef_ang_modelo, coef_lin_modelo):
        
        super().__init__(nome, simbolo, preco, ca0)
        etr.ca0_gl = ca0
        etr.con_gl = con
        etr.massa_molar_oxido = massa_molar_oxido
        etr.proporcao_estequiometrica = proporcao_estequiometrica #proporção estequiométrica entre o elemento e o seu óxido
        etr.a = coef_ang_modelo
        etr.b = coef_lin_modelo
        etr.ca0 = etr.ca0_gl * etr.proporcao_estequiometrica / etr.massa_molar_oxido
        etr.con = etr.con_gl * etr.proporcao_estequiometrica / etr.massa_molar_oxido
        etr.pureza_minima = pureza_minima
        
    def propriedades(etr): #o método '__dict__' ou 'vars(objeto)' fazem a mesma coisa, só que mais feio.
        
        print ('Nome: '
               + etr.nome
               + '\nConcentração da Alimentação Aquosa (g/L): '
               + str(etr.ca0_gl)
               + '\nConcentração da Alimentação Orgânica (g/L): '
               + str(etr.con_gl)
               + '\nMassa Molar (g/mol): '
               + str(etr.massa_molar_oxido)
               + '\nLog D =', etr.a, '* pH ', etr.b)
   
    def D (etr, H):
        
        D = H ** - etr.a * 10 ** etr.b        
        return D    
        
    def concentracoes_organico(etr, D, ca):
        
        co = D * ca        
        return co
    
    def conservacao_de_massa_aq(etr, ca_anterior, ca_atual, ca_posterior, h_atual, h_posterior, rao):
                
        cm_etr = (
                  ca_atual * (etr.D(h_atual) + rao)
                  - ca_posterior * etr.D(h_posterior)
                  - ca_anterior * rao
                  )
        
        return cm_etr
    
    def conservacao_de_massa_org(etr, ca_anterior, ca_atual, co_inicial, h_atual, rao):

        cm_etr = (
                  ca_atual * (etr.D(h_atual) + rao)
                  - co_inicial
                  - ca_anterior * rao
                  )
        
        return cm_etr

In [7]:
class Celula(Coisa_Fisica):
    
    def __init__(celula, nome, preco, numero, tempo_depreciacao = None):    

        super().__init__(nome, preco)
        celula.numero = numero
        celula.tempo_depreciacao = tempo_depreciacao
        
    def balanco_de_massa(celula, etr, cas_etr, cas_H, rao, n_celulas):
        
        if celula.numero == 1:
            bm_etr = etr.conservacao_de_massa_aq(etr.ca0, 
                                                 cas_etr['ca1'],
                                                 cas_etr['ca2'],
                                                 cas_H['ca1'],
                                                 cas_H['ca2'],
                                                 rao)
        elif celula.numero == n_celulas:
            bm_etr = etr.conservacao_de_massa_org(cas_etr['ca' + str(n_celulas - 1)],
                                                  cas_etr['ca' + str(n_celulas)],
                                                  etr.con,
                                                  cas_H['ca' + str(n_celulas)],
                                                  rao)
        else:
            bm_etr = etr.conservacao_de_massa_aq(cas_etr['ca' + str(celula.numero - 1)],
                                                 cas_etr['ca' + str(celula.numero)],
                                                 cas_etr['ca' + str(celula.numero + 1)],
                                                 cas_H['ca' + str(celula.numero)],
                                                 cas_H['ca' + str(celula.numero + 1)],
                                                 rao)
            
        return bm_etr
    
    def balanco_de_carga(celula, cas_H, lista_cas_etrs, K):
                
        etrs_totais_celula = 0
        
        for cas_etr in range(len(lista_cas_etrs)):
            etr_total_celula = lista_cas_etrs[cas_etr]['ca' + str(celula.numero)]
            etrs_totais_celula += etr_total_celula
            
        carga_total_protons_celula = cas_H['ca' + str(celula.numero)]
        carga_total_celula = 3 * etrs_totais_celula + carga_total_protons_celula
        
        bc = carga_total_celula - K
            
        return bc 

In [8]:
class Isoterma():

    def __init__ (isoterma, n_celulas, rao, lista_elementos):
        
        assert type(lista_elementos) == list
        assert type(lista_elementos[0]) == Proton
        
        isoterma.n_celulas = n_celulas
        isoterma.rao = rao
        isoterma.lista_elementos = lista_elementos
        
        if isoterma.lista_elementos[1].ca0 > 0:
            isoterma.tipo = 'Extração'
        else:
            isoterma.tipo = 'Lavagem'
        
    def junta_chutes_iniciais(isoterma):
        
        proton = isoterma.lista_elementos[0]
        lista_etrs = isoterma.lista_elementos[1:]
        
        if isoterma.tipo == 'Extração':
            chutes_H = proton.chutes_iniciais(isoterma.n_celulas, 1.2 * proton.ca0)        
            chutes_etrs = np.concatenate([etr.chutes_iniciais(isoterma.n_celulas, 0.001 * etr.ca0) for etr in lista_etrs])
            
        else:
            chutes_H = proton.chutes_iniciais(isoterma.n_celulas, 0.8 * proton.ca0)        
            chutes_etrs = np.concatenate([etr.chutes_iniciais(isoterma.n_celulas, 0.001 * etr.con) for etr in lista_etrs])
        chutes = np.concatenate((chutes_H, chutes_etrs))
        
        return chutes
        
    def junta_variaveis(isoterma, chutes):
        
        proton = isoterma.lista_elementos[0]
        chutes_H = chutes[0 : isoterma.n_celulas]
        
        cas_H = proton.concentracoes_aquoso(chutes_H)
        cas = [cas_H]        
                        
        for i in range(1, len(isoterma.lista_elementos)):            
            etr = isoterma.lista_elementos[i]            
            chutes_etr = chutes[i * isoterma.n_celulas : (i + 1) * isoterma.n_celulas]
            cas_etr = etr.concentracoes_aquoso(chutes_etr)
            cas.append(cas_etr)
            
        return cas
            
    def constante_de_carga(isoterma):
                
        proton = isoterma.lista_elementos[0]
        cargas_totais_etr = 0
        
        for etr in isoterma.lista_elementos[1:]:
            cargas_totais_etr += 3 * (etr.ca0)           
        
        K = cargas_totais_etr + proton.ca0
        
        return K 
        
    def equacoes_massa_carga(isoterma, chutes):
                
        rao = isoterma.rao
        n_celulas = isoterma.n_celulas
        cas_H = isoterma.junta_variaveis(chutes)[0]
        lista_cas_etrs = isoterma.junta_variaveis(chutes)[1:]
        K = isoterma.constante_de_carga()
        
        equacoes = []       
           
        for numero_da_celula in range(1, n_celulas + 1):             
            celula = Celula(None, None, numero_da_celula)
            
        #equações de balanço de massa        
            for cas_etrs in range(len(lista_cas_etrs)):
                cas_etr = lista_cas_etrs[cas_etrs]
                etr = isoterma.lista_elementos[cas_etrs + 1]
                bm_etr = celula.balanco_de_massa(etr, cas_etr, cas_H, rao, n_celulas)
                equacoes.append(bm_etr)
        
        #equações de balanço de carga       
            bc = celula.balanco_de_carga(cas_H, lista_cas_etrs, K)
            equacoes.append(bc)
                                
        return equacoes
    
    def resolve_equacoes_massa_carga(isoterma):

        resultados = fsolve(isoterma.equacoes_massa_carga, isoterma.junta_chutes_iniciais())
        return resultados
    
    def monta_tabela(isoterma, uso, excel = False):
        
        resultados = isoterma.resolve_equacoes_massa_carga()
        tabela_externa = pd.DataFrame({'Estágio Nº': np.arange(1, isoterma.n_celulas + 1)})
        tabela_interna = pd.DataFrame({'Estágio Nº': np.arange(1, isoterma.n_celulas + 1)})
        
        for i in range(len(isoterma.lista_elementos)):
            elemento = isoterma.lista_elementos[i]
            simbolo_elementar = elemento.simbolo
            concentracoes_aq_molar = (resultados[isoterma.n_celulas * i: isoterma.n_celulas * (i + 1)])
            
            if type(elemento) != Proton:                
                concentracoes_aq_gl = (concentracoes_aq_molar * elemento.massa_molar_oxido
                                       / elemento.proporcao_estequiometrica)
                D = elemento.D(tabela_externa['[' + isoterma.lista_elementos[0].simbolo + '](mol/L)'])
                concentracoes_org_molar = elemento.concentracoes_organico(D, concentracoes_aq_molar)
                concentracoes_org_gl = elemento.concentracoes_organico(D, concentracoes_aq_gl)                
                
                tabela_interna['[' + simbolo_elementar + ']aq(mol/L)'] = concentracoes_aq_molar
                tabela_externa['[' + simbolo_elementar + ']aq(g/L)'] = concentracoes_aq_gl
                tabela_interna['[' + simbolo_elementar + ']org(mol/L)'] = concentracoes_org_molar
                tabela_externa['[' + simbolo_elementar + ']org(g/L)'] = concentracoes_org_gl
                tabela_externa['D ' + simbolo_elementar] = D              
                
                if i > 1:                    
                    Beta = D / tabela_externa['D ' + isoterma.lista_elementos[i - 1].simbolo]
                    tabela_externa['Beta ' + simbolo_elementar + '/' + isoterma.lista_elementos[i - 1].simbolo] = Beta
            
            else:                
                tabela_interna['[' + simbolo_elementar + '](mol/L)'] = concentracoes_aq_molar
                tabela_externa['[' + simbolo_elementar + '](mol/L)'] = concentracoes_aq_molar
                tabela_externa['pH'] = elemento.pH(concentracoes_aq_molar)
        
        tabela_externa.set_index('Estágio Nº', inplace = True)
        tabela_interna.set_index('Estágio Nº', inplace = True)
        
        if excel:            
            tabela_externa.to_excel('Isoterma.xlsx',
                                    sheet_name = str((isoterma.rao, isoterma.n_celulas,
                                                      isoterma.lista_elementos[0].pH_inicial)))            
        if uso == 'interno':
            return tabela_interna
        else:
            return tabela_externa
    
    def resumo(isoterma):
        
        tabela = isoterma.monta_tabela(uso = 'interno')
        
        tabela_resumida = pd.DataFrame(columns = [etr.simbolo for etr in isoterma.lista_elementos[1:]])
        tabela_resumida.loc['Alimentação aq. (mol/L)'] = [etr.ca0 for etr in isoterma.lista_elementos[1:]]
        tabela_resumida.loc['Alimentação org. (mol/L)'] = [etr.con for etr in isoterma.lista_elementos[1:]]
        tabela_resumida.loc['Rafinado (mol/L)'] = np.array(tabela.iloc[-1, 1: :2])
        tabela_resumida.loc['Carregado Orgânico (mol/L)'] = np.array(tabela.iloc[0, 2: :2])
        tabela_resumida.loc['Composição Rafinado (%)'] = (tabela_resumida.loc['Rafinado (mol/L)'] * 100 
                                                          / tabela_resumida.loc['Rafinado (mol/L)'].sum())        
        tabela_resumida.loc['Composição org. Carregado (%)'] = (tabela_resumida.loc['Carregado Orgânico (mol/L)']
                                                                * 100 / (tabela_resumida.loc['Carregado Orgânico (mol/L)']
                                                                .sum()))
        if isoterma.tipo == 'Extração':
            tabela_resumida.loc['Recuperação Rafinado (%)'] = (tabela_resumida.loc['Rafinado (mol/L)'] * 100
                                                               / tabela_resumida.loc['Alimentação aq. (mol/L)'])           
        else:
            tabela_resumida.loc['Recuperação Rafinado (%)'] = ((tabela_resumida.loc['Alimentação org. (mol/L)'] 
                                                                - tabela_resumida.loc['Carregado Orgânico (mol/L)'])
                                                                * 100 / tabela_resumida.loc['Alimentação org. (mol/L)'])
        
        tabela_resumida.loc['Recuperação Orgânico (%)'] = 100 - tabela_resumida.loc['Recuperação Rafinado (%)']
        
        return tabela_resumida
    
    def monta_grafico(isoterma):
        
        tabela = isoterma.monta_tabela(uso = 'externo')
        
        for elemento in isoterma.lista_elementos[1:]:
            posicao_na_lista = 1
            reta_operacional = np.concatenate(
                                              (
                                               [elemento.ca0_gl],
                                               np.array(tabela['[' + elemento.simbolo + ']aq(g/L)']),
                                               np.array(tabela['[' + elemento.simbolo + ']org(g/L)']),
                                               [elemento.con_gl]
                                               )
                                              ).reshape(2, isoterma.n_celulas + 1)
            
            estagios_aquoso = [reta_operacional[0, 0]]
            for n in range(1, isoterma.n_celulas + 1):                               
                estagios_aquoso += 2 * [reta_operacional[0, n]]
                
            estagios_organico = []
            for n in range(0, isoterma.n_celulas):                               
                estagios_organico += 2 * [reta_operacional[1, n]]
            estagios_organico += [reta_operacional[1, -1]]
                        
            fig = plt.figure()
            axes = fig.add_axes([0, posicao_na_lista - 1.2, 1, 1])
            axes.plot(estagios_aquoso, estagios_organico, '.-', markersize = 8, label = "Estágios")
            axes.plot(reta_operacional[0, :], reta_operacional[1, :], '.-', markersize = 8, label = 'Reta de Operação')
            axes.plot(np.array(tabela['[' + elemento.simbolo + ']aq(g/L)']),
                      np.array(tabela['[' + elemento.simbolo + ']org(g/L)']),
                      '.-', markersize = 8, label = 'Reta de Equilíbrio')
            axes.set_xlabel('Aquoso')
            axes.set_ylabel('Orgânico')
            axes.set_ylim([0, None])
            axes.set_xlim([0, None])
            axes.legend(loc = 0)
            axes.set_title('Isoterma de ' + isoterma.tipo + ' de ' + elemento.nome)            

In [9]:
class Operacao():
    
    def __init__(operacao, lista_etrs,
                 extratante, conc_ext, solv_org, acido, celula,
                 n_min_celulas = 2, n_max_celulas = 10,
                 rao_min = 0.5, rao_max = 2.0,
                 pH_min = 1, pH_max = 1.5):
        
        operacao.lista_etrs = lista_etrs
        operacao.extratante = extratante
        operacao.conc_ext = conc_ext
        operacao.solv_org = solv_org
        operacao.acido = acido
        operacao.celula = celula
        operacao.n_min_celulas = n_min_celulas
        operacao.n_max_celulas = n_max_celulas
        operacao.rao_min = rao_min
        operacao.rao_max = rao_max
        operacao.pH_min = pH_min
        operacao.pH_max = pH_max
        operacao.tempo_projeto = operacao.celula.tempo_depreciacao
    
    def testa_varias_isotermas(operacao, excel = False):
        
        global H
        H = Proton('Próton', 'H+', None, 10 ** - operacao.pH_min)
        global isoterma
        isoterma = Isoterma(operacao.n_min_celulas, 0.5, [H] + operacao.lista_etrs)
        resumo_exemplo = isoterma.resumo()
        colunas_do_df = []
        
        for indice in range(len(resumo_exemplo)): 
            for elemento in operacao.lista_etrs:
                indice_do_df = elemento.simbolo + ' ' + resumo_exemplo.index[indice]
                colunas_do_df.append(indice_do_df)
                
        guarda_condicoes = pd.DataFrame(columns = (['Operação', 'Extratante', 'Nº Células', 'Razão A/O', 'pH inicial']
                                                   + colunas_do_df))

        for n_celulas in range(operacao.n_min_celulas, operacao.n_max_celulas + 1):
            isoterma.n_celulas = n_celulas
            
            for rao in np.linspace(operacao.rao_min, operacao.rao_max, 
                                   int((operacao.rao_max - operacao.rao_min) * 10 + 1)):
                isoterma.rao = rao
                
                for ca0 in np.logspace(- operacao.pH_min, - operacao.pH_max, 
                                       int((operacao.pH_max - operacao.pH_min) * 10) + 1):                  
                    H.ca0 = ca0
                    H.pH_inicial = -np.log10(H.ca0)
                    resumo = isoterma.resumo()
                    
                    if (
                        (
                         isoterma.tipo == 'Extração' and resumo.loc['Composição Rafinado (%)'][: -1].sum() >= 99.5
                         ) or
                            (
                             isoterma.tipo == 'Lavagem' and 
                                (
                                 resumo.loc['Composição Rafinado (%)'][: -1].sum() >= 99 or
                                 resumo.loc['Composição org. Carregado (%)'][-1] >= operacao.lista_etrs[-1].pureza_minima
                                 )
                             )
                        ):
                        
                            condicao = [isoterma.tipo, 
                                        operacao.extratante.nome +' '+ str(int(100 * operacao.conc_ext)) + '%',
                                        n_celulas, rao, H.pH_inicial]
                            for linha in range(len(resumo)):
                                for etr in range(len(operacao.lista_etrs)):
                                    condicao.append(resumo.iloc[linha, etr])
                            condicao = pd.Series(condicao, index = guarda_condicoes.columns)
                            guarda_condicoes = guarda_condicoes.append(condicao, ignore_index = True)
        
        if excel:
            guarda_condicoes.sort_values(by = [operacao.lista_etrs[0].simbolo + ' Recuperação Rafinado (%)'],
                                         ascending = False, inplace = True)
            guarda_condicoes.to_excel('Melhores Condições.xlsx', sheet_name = 'Melhores Condições')
        
        return guarda_condicoes          
            
    def viabilidade_economica(operacao, condicoes, kg_etr_produzido, excel = False, nome_planilha = None):

        inv_condicoes = []
        cpv_condicoes = []
        receita_condicoes = [kg_etr_produzido * operacao.tempo_projeto * operacao.lista_etrs[1].preco
                             for i in range(len(condicoes))]
        
        for i in range(len(condicoes)):
            
            etr_total_gl = np.array([(condicoes.iloc[i][etr.simbolo + ' Rafinado (mol/L)'] 
                                      * etr.massa_molar_oxido
                                      / etr.proporcao_estequiometrica)
                                    for etr in operacao.lista_etrs[: -1]]).sum()
            
            vol_aq = kg_etr_produzido * 1000 / etr_total_gl
            vol_org = vol_aq / condicoes.iloc[i]['Razão A/O']
            vol_ext = vol_org * operacao.conc_ext / operacao.extratante.pureza
            vol_solv = vol_org - vol_ext            
            conc_molar_acido = operacao.acido.concentracao_mm * operacao.acido.densidade / operacao.acido.MM
            vol_acido = vol_aq * 10 ** - condicoes.iloc[i]['pH inicial'] / conc_molar_acido
            
            inv_ext = operacao.extratante.preco * vol_ext
            inv_solv = operacao.solv_org.preco * vol_solv            
            inv_cels = operacao.celula.preco * condicoes.iloc[i]['Nº Células']
            
            cpv_acido = operacao.acido.preco * vol_acido * operacao.acido.densidade
            cpv_ext = inv_ext * operacao.extratante.taxa_perda
            cpv_solv = inv_solv * operacao.solv_org.taxa_perda            
            cpv = cpv_acido + cpv_ext + cpv_solv
            cpv_projeto = cpv * operacao.tempo_projeto
            
            inv_giro = cpv
            inv_projeto = inv_ext + inv_solv + inv_cels + inv_giro
                                  
            inv_condicoes.append(inv_projeto)
            cpv_condicoes.append(cpv_projeto)
        
        condicoes['Receita'] = receita_condicoes
        condicoes['CPV'] = cpv_condicoes
        condicoes['CapEx'] = inv_condicoes        
        condicoes['Saldo de Caixa'] = condicoes['Receita'] - condicoes['CPV'] - condicoes['CapEx']
        
        condicoes.sort_values(by = ['Saldo de Caixa'], ascending = False, inplace = True)
        
        if excel:
            condicoes.to_excel(nome_planilha + '.xlsx', sheet_name = 'Melhores Condições')
        
        return condicoes    
        
    def atualiza_parametros(operacao, parametros_da_operacao, etapa_operacao):
        
        parametros_op = parametros_da_operacao[etapa_operacao + 1]        
        operacao.n_min_celulas = parametros_op[0]
        operacao.n_max_celulas = parametros_op[1]
        operacao.rao_min = parametros_op[2]
        operacao.rao_max = parametros_op[3]
        operacao.pH_min = parametros_op[4]
        operacao.pH_max = parametros_op[5]
        
    def atualiza_correntes(operacao, condicao):
        
        for j in range(len(operacao.lista_etrs)):
            etr = operacao.lista_etrs[j]                    
            if isoterma.tipo == 'Extração':
                etr.con = condicao.loc[etr.simbolo + ' Carregado Orgânico (mol/L)']                    
                etr.ca0 = 0                    
            else:
                etr.ca0 = condicao.loc[etr.simbolo + ' Carregado Orgânico (mol/L)']  
                etr.con = 0
                
            etr.ca0_gl = etr.ca0 * etr.massa_molar_oxido / etr.proporcao_estequiometrica
            etr.con_gl = etr.con * etr.massa_molar_oxido / etr.proporcao_estequiometrica 
                      
    def liga_isotermas(operacao, parametros_da_operacao, excel = False):
        #necessita atualização
        
        assert type(parametros_da_operacao) == list
        assert type((parametros_da_operacao)[0]) == list
        assert type((parametros_da_operacao)[1]) == list
        assert type((parametros_da_operacao)[2]) == list
                
        operacao.lista_etrs = parametros_da_operacao[0]
        operacao.extratante = parametros_da_operacao[1][0]
        operacao.conc_ext = parametros_da_operacao[1][1]
        operacao.solv_org = parametros_da_operacao[1][2]
        operacao.acido = parametros_da_operacao[1][3]
        operacao.celula = parametros_da_operacao[1][4]
        
        etapa_operacao = 1
        operacao.atualiza_parametros(parametros_da_operacao, etapa_operacao)
        condicoes_guardadas1 = operacao.testa_varias_isotermas()
        kg_etr_produzido = 1 #volume da fase aquosa da etapa de extração
        condicoes_guardadas1 = operacao.viabilidade_economica(condicoes_guardadas1, kg_etr_produzido)
        
        condicoes_finais = pd.DataFrame(columns = condicoes_guardadas1.columns)
        condicoes_finais = pd.concat([condicoes_finais] * 2, axis = 1)
        
        etapa_operacao += 1
        
        for condicao1 in range(len(condicoes_guardadas1)):
            condicao_guardada1 = condicoes_guardadas1.iloc[condicao1]
            rao1 = condicao_guardada1['Razão A/O']
            operacao.atualiza_correntes(condicao_guardada1)                
            operacao.atualiza_parametros(parametros_da_operacao, etapa_operacao)
            condicoes_guardadas2 = operacao.testa_varias_isotermas()            
            condicoes_guardadas2 = operacao.viabilidade_economica(condicoes_guardadas2, None, rao1)

            for condicao2 in range(len(condicoes_guardadas2)):
                condicao_guardada2 = condicoes_guardadas2.iloc[condicao2]
                condicao_final = pd.concat([condicao_guardada1, condicao_guardada2])
                condicoes_finais = condicoes_finais.append(condicao_final, ignore_index = True)
        
        for i in range(len(condicoes_finais)):
            condicoes_finais['Gastos Totais da Operação'] = condicoes_finais['Gastos da Etapa'].iloc[i].sum()
        
        if len(condicoes_finais) > 1:
            condicoes_finais.sort_values(by = ['Gastos Totais da Operação'], inplace = True)
        
        if excel:
            condicoes_finais.T.to_excel('Condições Finais.xlsx', sheet_name = 'Melhores Condições')

        return condicoes_finais
    
def testa_varias_operacoes(lista_operacoes, kg_etr_produzido, excel = False, nome_planilha = None):
        
    assert type(lista_operacoes) == list
    condicoes = lista_operacoes[0].viabilidade_economica(lista_operacoes[0].testa_varias_isotermas(), kg_etr_produzido)
        
    for operacao in lista_operacoes[1:]:
        condicoes = pd.concat([operacao.viabilidade_economica(operacao.testa_varias_isotermas(), kg_etr_produzido),
                               condicoes])
    
    condicoes = condicoes.drop(condicoes.iloc[:, 5:29], axis = 1)
    condicoes.sort_values(by = ['Saldo de Caixa'], ascending = False, inplace = True)    
            
    if excel:
        condicoes.to_excel('Ranking Extratantes.xlsx', sheet_name = 'Melhores Condições')
        
    return condicoes            

In [10]:
hcl = Substancia('HCl', 1, {'densidade': 1179, 'concentracao_mm': 0.37, 'MM': 36.5}) #preço em R$/kg
dehpa = Substancia('D2EHPA', 15.52, {'densidade': 0.97, 'pureza': 0.95, 'taxa_perda' : 0.1}) #preço em R$/L
p507 = Substancia('P507', 80.75, {'densidade': 0.95, 'pureza': 0.95, 'taxa_perda' : 0.1}) #preço em R$/L
cyanex272 = Substancia('Cyanex 272', 208.5, {'densidade': 0.93, 'pureza': 0.9, 'taxa_perda' : 0.1}) #preço em R$/L
isoparafina = Substancia('Isoparafina', 8.9, {'pureza': 1, 'taxa_perda' : 0.1}) #preço em R$/L
mixset = Celula('Mixer-Settler', 54200, None, 10) #preco em R$/unit.

In [18]:
# Isotermas de lavagem parecem passíveis de otimização por chutes mais precisos

disprosio = ETR('Disprósio', 'Dy', 600, 99.5, 0, 3.19, 372.998, 2, 3.22552, -0.99999)
holmio = ETR('Hólmio', 'Ho', 40, 99.9, 0, 6.71, 377.858, 2, 3.05008, -0.68760)
H = Proton('Próton', 'H+', None, 0.805)
isoterma = Isoterma(20, 0.787, [H, disprosio, holmio])
operacao = Operacao([disprosio, holmio], p507, 0.26, isoparafina, hcl, mixset, 30, 30, 1, 1, 0.3, 0.3)

In [12]:
# D2EHPA 10%

praseodimio1 = ETR('Praseodímio', 'Pr', 75, 99.5, 0.480, 0, 1021.44, 6, 2.84, -3.6807)
neodimio1 = ETR('Neodímio', 'Nd', 75, 99.5, 12.808, 0, 336.48, 2, 1.0539, -1.7022)
samario1 = ETR('Samário', 'Sm', 10, 99.9, 0.923, 0, 348.72, 2, 1.5262, -1.1515)
operacao1 = Operacao([praseodimio1, neodimio1, samario1], dehpa, 0.1, isoparafina, hcl, mixset, 2, 10, 0.5, 2, 1, 2)
H2 = Proton('Próton', 'H+', None, 10**-2)
isoterma = Isoterma(2, 0.8, [H2, praseodimio1, neodimio1, samario1])

In [13]:
# D2EHPA 6%

praseodimio2 = ETR('Praseodímio', 'Pr', 75, 99.5, 0.480, 0, 1021.44, 6, 0.4026, -1.5661)
neodimio2 = ETR('Neodímio', 'Nd', 75, 99.5, 12.808, 0, 336.48, 2, 0.8569, -1.9035)
samario2 = ETR('Samário', 'Sm', 10, 99.9, 0.923, 0, 348.72, 2, 1.1924, -1.2654)
operacao2 = Operacao([praseodimio2, neodimio2, samario2], dehpa, 0.06, isoparafina, hcl, mixset, 2, 10, 0.5, 2, 1, 2)
H2 = Proton('Próton', 'H+', None, 10**-2)
isoterma = Isoterma(2, 0.8, [H2, praseodimio2, neodimio2, samario2])

In [14]:
# P507 10%

praseodimio3 = ETR('Praseodímio', 'Pr', 75, 99.5, 0.480, 0, 1021.44, 6, 0.7487, -2.3629)
neodimio3 = ETR('Neodímio', 'Nd', 75, 99.5, 12.808, 0, 336.48, 2, 1.6106, -3.3279)
samario3 = ETR('Samário', 'Sm', 10, 99.9, 0.923, 0, 348.72, 2, 2.213, -2.9758)
operacao3 = Operacao([praseodimio3, neodimio3, samario3], p507, 0.1, isoparafina, hcl, mixset, 2, 10, 0.5, 2.5, 1.2, 2)

In [15]:
# P507 6%

praseodimio4 = ETR('Praseodímio', 'Pr', 75, 99.5, 0.480, 0, 1021.44, 6, 6.5856, -11.441)
neodimio4 = ETR('Neodímio', 'Nd', 75, 99.5, 12.808, 0, 336.48, 2, 1.5074, -3.5508)
samario4 = ETR('Samário', 'Sm', 10, 99.9, 0.923, 0, 348.72, 2, 2.1979, -3.4924)
operacao4 = Operacao([praseodimio4, neodimio4, samario4], p507, 0.06, isoparafina, hcl, mixset, 2, 10, 0.5, 2.5, 1.2, 2)

In [16]:
# Cyanex 572 10%

praseodimio5 = ETR('Praseodímio', 'Pr', 75, 99.5, 0.480, 0, 1021.44, 6, 0.393, -2.2876)
neodimio5 = ETR('Neodímio', 'Nd', 75, 99.5, 12.808, 0, 336.48, 2, 1.5592, -3.976)
samario5 = ETR('Samário', 'Sm', 10, 99.9, 0.923, 0, 348.72, 2, 1.9341, -3.2532)
operacao5 = Operacao([praseodimio5, neodimio5, samario5], cyanex272, 0.1, isoparafina, hcl, mixset, 2, 10, 0.5, 2.5, 1.5, 2.5)